# Wrangle OpenStreetMap data
## Udacity Data Analyst Nanodegree - Project 3

### Introduction

#### Map
* Cologne, http://www.openstreetmap.org/#map=11/50.9387/6.8740
* Before moving to Milan, Italy where I currently live, Cologne has been my home for several years. I know this city quite well and I am curious to improve its OSM data

### Resources
* Udacity course materials
* [Python 3 documentation](https://docs.python.org/3/)
* [MongoDB documentation](https://docs.mongodb.com/manual/)
* [MongoDB driver documentation](https://docs.mongodb.com/ecosystem/drivers/python/)
* [Markdown documentation](https://daringfireball.net/projects/markdown/syntax)

#### Generate sample OSM file

In [ ]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "cologne_germany.osm"  # Replace this with your osm file
SAMPLE_FILE = "cologne_germany_sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    """
    Changed output of write to byte objects in order to work with Python 3.x
    
    Reference:
    http://stackoverflow.com/questions/33054527/python-3-5-typeerror-a-bytes-like-object-is-required-not-str
    """
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>')

### Data exploration

In [1]:
# setup environment
import xml.etree.ElementTree as ET
import pprint
import re

#### Helper functions

In [3]:
def head_dict(input_dict, n=20):
    """
    Returns the first n (default=20) items of a dict
    """
    return dict(list(sorted(input_dict.items()))[0:n])

#### Get overview of tags

In [4]:
def get_tag_counts(file="cologne_germany_sample.osm"):
    """
    Given a valid XML input file, the function returns a dict of tags and their respective counts
    """
    # create tags dict
    tag_counts = {}
    
    # open file
    with open(file, "r", encoding="utf8") as f:
        
        # loop over file
        for event, elem in ET.iterparse(f):
        
            # check if tag is in dict
            if elem.tag not in tag_counts.keys():
    
                # if not, add tag as new key
                tag_counts[elem.tag] = 1
        
            # if so... 
            else:
                
                #increase count of identifed tag
                tag_counts[elem.tag] += 1

    return tag_counts

In [5]:
get_tag_counts()

{'member': 4594,
 'nd': 235134,
 'node': 156597,
 'osm': 1,
 'relation': 353,
 'tag': 153299,
 'way': 30566}

#### Get overview ot tag keys and values

In [6]:
def get_tag_types(file="cologne_germany_sample.osm"):
    """
    Given a valid XML input file, the function returns a dict of keys and corresponding counts of the "tag" attribute
    """
    # create tags dict
    tag_types = {}
    
    # open file
    with open(file, "r", encoding="utf8") as f:
        
        # loop over file
        for event, elem in ET.iterparse(f):
            
            # inspect only "tag" elements
            if elem.tag == "tag":
        
                # loop over "tag" elements
                for tag in elem.iter("tag"):
                
                    # check if tag key not in tags_types dict
                    if tag.attrib["k"] not in tag_types.keys():
                        
                        # if not add key with count 1
                        tag_types[tag.attrib["k"]] = 1
                    
                    else:
                        
                        # if so increase count
                        tag_types[tag.attrib["k"]] += 1      
    
    return tag_types

In [10]:
tag_types = get_tag_types()

In [18]:
pprint.pprint(tag_types)

{'Denkmalnummer': 2,
 'FIXME': 39,
 'TMC:cid_58:tabcd_1:Class': 46,
 'TMC:cid_58:tabcd_1:Direction': 15,
 'TMC:cid_58:tabcd_1:LCLversion': 46,
 'TMC:cid_58:tabcd_1:LocationCode': 46,
 'TMC:cid_58:tabcd_1:NextLocationCode': 28,
 'TMC:cid_58:tabcd_1:PrevLocationCode': 28,
 'TMC:cid_58:tabcd_1:TypeName': 1,
 'TMC:cid_58:tabcd_1:TypeName:loc': 1,
 'VRS:gemeinde': 196,
 'VRS:name': 57,
 'VRS:ortsteil': 196,
 'VRS:ref': 195,
 'abandoned': 3,
 'access': 712,
 'addr:city': 13392,
 'addr:country': 13182,
 'addr:district': 1,
 'addr:housename': 18,
 'addr:housenumber': 13337,
 'addr:postcode': 13351,
 'addr:street': 13395,
 'addr:suburb': 246,
 'admin_centre:postal_code': 1,
 'admin_level': 15,
 'advertising': 10,
 'aerialway': 3,
 'aerialway:heating': 2,
 'aerialway:occupancy': 2,
 'aeroway': 2,
 'agricultural': 4,
 'alt_name': 11,
 'amenity': 1414,
 'animal': 2,
 'apartments': 1,
 'architect': 21,
 'area': 76,
 'area:highway': 1,
 'artist': 1,
 'artist_name': 3,
 'artwork_type': 5,
 'asb': 5,


In [20]:
head_dict(tag_types)

{'Denkmalnummer': 2,
 'FIXME': 39,
 'TMC:cid_58:tabcd_1:Class': 46,
 'TMC:cid_58:tabcd_1:Direction': 15,
 'TMC:cid_58:tabcd_1:LCLversion': 46,
 'TMC:cid_58:tabcd_1:LocationCode': 46,
 'TMC:cid_58:tabcd_1:NextLocationCode': 28,
 'TMC:cid_58:tabcd_1:PrevLocationCode': 28,
 'TMC:cid_58:tabcd_1:TypeName': 1,
 'TMC:cid_58:tabcd_1:TypeName:loc': 1,
 'VRS:gemeinde': 196,
 'VRS:name': 57,
 'VRS:ortsteil': 196,
 'VRS:ref': 195,
 'abandoned': 3,
 'access': 712,
 'addr:city': 13392,
 'addr:country': 13182,
 'addr:district': 1,
 'addr:housename': 18}

#### Explore "fixme" and "fixed" tag keys

In [24]:
def get_tag_key(file="cologne_germany_sample.osm", key="FIXME"):
    """
    Given a valid XML input file, the function returns a list of values for the corresponding key of the "tag" attribute
    """
    # create tags dict
    tag_keys = []
    
    # open file
    with open(file, "r", encoding="utf8") as f:
        
        # loop over file
        for event, elem in ET.iterparse(f):
            
            # inspect only "tag" elements
            if elem.tag == "tag":
        
                # loop over "tag" elements
                for tag in elem.iter("tag"):
                
                    # check if tag key not in tags_types dict
                    if tag.attrib["k"] == key:
                        
                        # if not add key with count 1
                        tag_keys.append(tag.attrib["v"])
                    
                    else:
                        continue     
    
    return set(tag_keys)

In [28]:
tag_key_fixme = get_tag_key()

In [30]:
pprint.pprint(tag_key_fixme)

{'Bitte Details ergänzen',
 'Bitte Existenz des Defi prüfen.',
 'Bitte Gebäude, Gebäudeteil oder Eingang zuordnen',
 'Bitte näher bezeichnen. barrier=fence?',
 'Diese Landuse Relation sollte man verkleinern',
 'Gebäudeumrisse prüfen',
 'Verbindung?',
 'auch Eingang Ehrenstraße 2',
 'bessere Beschreibung erforderlich',
 'bitte Gebäude oder Gebäudeteil zuordnen',
 'bitte Gebäude zuordnen',
 'bitte Gebäude zuordnen (auf Bild in 12/2013 nicht vorhanden)',
 'bitte Gebäude, Gebäudeteil oder Eingang zuordnen',
 'bitte Gebäudeeingang oder Gebäudeteil zuordnen',
 'bitte Gebäudeeingang zuordnen',
 'bitte Gebäudeingang oder Gebäudeteil zuordnen',
 'bitte Hauseingang bzw. Gebäudeteil zuordnen',
 'bitte Hauseingang oder Gebäudeteil zuordnen',
 'bitte Name und Details ergänzen',
 'bitte Nutzungsart ergänzen',
 'bitte genau zuordnen',
 'bitte richtig zuordnen',
 'ist dieser Abschnitt Einbahnstraße?',
 'lage geschätzt, Juni 2013',
 'landuse=grass für diese großen Planzkästen scheint mir etwas overdres

In [137]:
get_tag_key(key="fixed")

{'Amsterdamer Straße ist stadteinwärts die Fortsetzung der Industriestr. (Trunk)  und liegt auf der optimalen Route für den Fernverkehr aus der Innenstadt in Richtung Ruhrgebiet, Hannover, Bremen, Berlin und Hamburg. Deshalb als primary road kennzeichnen!',
 'Amsterdamer Straße ist stadteinwärts die Fortsetzung der Industriestr. (Trunk) und liegt auf der optimalen Route für den Fernverkehr aus der Innenstadt in Richtung Ruhrgebiet, Hannover, Bremen, Berlin und Hamburg. Deshalb als primary road kennzeichnen!',
 'Auf B 59 stadteinwärts wird Verkehr Ri. Zentrum nach links auf die Äußere Kanalstr. gewiesen (wegen Verkehrsberuhigung) To Do: Steht der Wegweiser noch? Wenn ja, für fixme-Abschnitte highway=secondary setzen, da keine Fern- u. Regionalnetz-Funktion mehr!',
 'Auf B 59 stadteinwärts wird Verkehr Ri. Zentrum nach links auf die Äußere Kanalstr. gewiesen. To Do: Steht der Wegweiser noch? Wenn ja, Subbelrather Str. als Parallele zur Venloer Str. (verkehrsberuhigt) regionale Netzfunkti

#### Explore other interresting tags

In [130]:
get_tag_key(key="addr:district")

{'Porz'}

In [131]:
get_tag_key(key="addr:suburb")

{'Brück',
 'Buchforst',
 'Deutz',
 'Grengel',
 'Kalk',
 'Lindenthal',
 'Mülheim',
 'Nippes',
 'Ostheim',
 'Poll',
 'Sielsdorf',
 'Urbach',
 'Vingst'}

In [133]:
get_tag_key(key="addr:housename")

{'Bayburt Kulturverein',
 'Bürgerhaus Stollwerck',
 'C103',
 'Caritas Hospiz Johannes-Nepomuk-Haus',
 'Doc-PT Praxis für Innere- und Allgemeinmedizin',
 'Erik Wickberg-Haus',
 'Feilenhof',
 'HERZBERGMEDIA',
 'Jüdisches Wohlfahrtszentrum',
 'Kartonagenfabrik Seybold',
 'Mo-Fr 09:30-18:30; Sa 09:30-16:00',
 'Post Office',
 'Raderthalgürtel',
 'Schaltwerk',
 'SkinWorks',
 'TrauerHaus Müschenborn',
 'Vereinsheim ESV Olympia Köln e.V.',
 'Villa Hahnenburg'}

In [32]:
get_tag_key(key="addr:country")

{'DE'}

In [33]:
get_tag_key(key="addr:city")

{'Bergisch Gladbach', 'Hürth', 'Köln', 'Köln Rath/Heumar', 'Köln-Nippes'}

In [35]:
tag_key_street = get_tag_key(key="addr:street")

In [36]:
pprint.pprint(tag_key_street)

{'Aachener Straße',
 'Abshofstraße',
 'Abtstraße',
 'Achterstraße',
 'Ackerstraße',
 'Ackerwinde',
 'Adalbertstraße',
 'Adam-Linden-Straße',
 'Adam-Stegerwald-Straße',
 'Adamsstraße',
 'Adelheidisstraße',
 'Adlerstraße',
 'Adlerweg',
 'Adolf-Fischer-Straße',
 'Adolf-Grimme-Allee',
 'Adolf-Kober-Straße',
 'Adolf-Menzel-Straße',
 'Adolphstraße',
 'Adrian-Kiels-Straße',
 'Aduchtstraße',
 'Aegidienberger Straße',
 'Aeltgen-Dünwald-Straße',
 'Afrastraße',
 'Aggerstraße',
 'Agrippastraße',
 'Agrippinawerft',
 'Ahornstraße',
 'Ahornweg',
 'Ahrstraße',
 'Ahrweg',
 'Ahrweilerstraße',
 'Akazienstraße',
 'Akazienweg',
 'Akeleiweg',
 'Alarichstraße',
 'Alban-Berg-Weg',
 'Albermannstraße',
 'Albert-Schneider-Straße',
 'Albert-Schweitzer-Straße',
 'Albertus-Magnus-Weg',
 'Albertusstraße',
 'Alemannenstraße',
 'Alexander-Petöfi-Platz',
 'Alexe-Altenkirch-Straße',
 'Alexianerstraße',
 'Alfred-Delp-Straße',
 'Alfred-Nobel-Straße',
 'Alfred-Schütte-Allee',
 'Alfred-Werner-Straße',
 'Alfterstraße',
 'Ali

In [31]:
get_tag_key(key="addr:postcode")

{'50354',
 '50667',
 '50668',
 '50670',
 '50672',
 '50674',
 '50676',
 '50677',
 '50678',
 '50679',
 '50733',
 '50735',
 '50737',
 '50739',
 '50765',
 '50767',
 '50823',
 '50825',
 '50827',
 '50829',
 '50858',
 '50859',
 '50931',
 '50933',
 '50935',
 '50937',
 '50939',
 '50968',
 '50969',
 '50996',
 '50997',
 '50999',
 '51061',
 '51063',
 '51065',
 '51067',
 '51069',
 '51103',
 '51105',
 '51107',
 '51109',
 '51143',
 '51145',
 '51147',
 '51149',
 '51427',
 '51467',
 '51469'}

In [135]:
get_tag_key(key="alt_name")

{'Chaussée Brunehaut (BE, F)',
 'Deutz-Thermalbad',
 'Dorint Sofitel An der Messe',
 'Entschlafen der Gottesmutter',
 'Gymnasium - Kaiserin-Theophanu-Schule',
 'Haus der Begegnung',
 'Kölner Seilbahn',
 'LVR-Klinik Forensische Psychiatrie',
 'Rhine Route - part Germany',
 'Stotzheim'}

In [138]:
get_tag_key(key="information")

{'board', 'guidepost', 'hikingmap', 'map', 'nature'}

In [139]:
get_tag_key(key="postal_code")

{'50668',
 '50672',
 '50674',
 '50733',
 '50765',
 '50767',
 '50935',
 '50937',
 '50996',
 '51061',
 '51063',
 '51065',
 '51067',
 '51069',
 '51103',
 '51109',
 '51143,51145',
 '51145',
 '51147'}

#### Audit tag keys

In [80]:
# complie regular expressions
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n\-]')

In [81]:
def get_audit_tags(file="cologne_germany_sample.osm"):
    """
    tbd
    """
    # create tags dict
    audit_tags = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    problemchars_list = []
    
    # open file
    with open(file, "r", encoding="utf8") as f:
        
        # loop over file
        for _ , elem in ET.iterparse(f):
    
            if elem.tag == "tag":
        
                # loop over tags of element
                for tag in elem.iter("tag"):
            
                    # check for lower
                    if re.search(lower, tag.attrib["k"]):
        
                        # increase count
                        audit_tags["lower"] += 1
        
                    # check for lower_colon
                    elif re.search(lower_colon, tag.attrib["k"]):
        
                        # increase count
                        audit_tags["lower_colon"] += 1
        
                    # check for problemchars
                    elif re.search(problemchars, tag.attrib["k"]):
                        
                        # add value to problemchars list
                        problemchars_list.append(tag.attrib["k"])
                        
                        # increase count
                        audit_tags["problemchars"] += 1
                  
        
                # else assign other
                else:
                    audit_tags["other"] += 1
        
    return audit_tags, set(problemchars_list)

In [82]:
_ , problemchars = get_audit_tags()

In [83]:
problemchars

{'step.condition',
 'step.height',
 'step.length',
 'strassen-nrw:abs',
 'surface.material'}

#### Fix problemchars

In [88]:
def fix_problemchars(file="cologne_germany_sample.osm", problemchars=problemchars, specific_issue=None, specific_fix=None):
    """
    tbd
    """
    fixed_dict = {}
    
    with open(file, "r", encoding="utf8") as f:
        
        # loop over file
        for _ , elem in ET.iterparse(f):
    
            if elem.tag == "tag":
        
                # loop over tags of element
                for tag in elem.iter("tag"):
                    
                    # identify problemchars
                    if tag.attrib["k"] in problemchars:
                        
                        # check for specific issue
                        if tag.attrib["k"] == specific_issue:
                            
                            # fix specific issue
                            fixed_dict[tag.attrib["k"]] = tag.attrib["k"].replace("-", specific_fix)
                        
                        # check for hiphen
                        elif re.search(re.compile(r'\-'), tag.attrib["k"]):
                                
                                # fix hiphen
                                fixed_dict[tag.attrib["k"]] = tag.attrib["k"].replace("-", "_")
                        
                        # check for dot
                        elif re.search(re.compile(r'\.'), tag.attrib["k"]):
                            
                                # fix dot
                                fixed_dict[tag.attrib["k"]] = tag.attrib["k"].replace(".", ":")
                                
                        
                        # keep track of changes
                        fixed_dict[tag.attrib["k"]] = tag.attrib["k"].replace(".", ":")
                        
                        # fix problemchars by replacing . with :
                        #tag.attrib["k"] = tag.attrib["k"].replace(".", ":")

    return fixed_dict

In [89]:
fix_problemchars(specific_issue="surface.material", specific_fix="_")

{'step.condition': 'step:condition',
 'step.height': 'step:height',
 'step.length': 'step:length',
 'strassen-nrw:abs': 'strassen-nrw:abs',
 'surface.material': 'surface:material'}

#### Fix city

In [ ]:
target_pattern = re.complile()

#### Prepare XML data for ingest into MongoDB

In [ ]:
# tbd

### Problems encountered
tbd

### Data overview
tbd

### Additional ideas
tbd

### Conclusion
tbd